<a href="https://colab.research.google.com/github/bibxo/sandlab/blob/main/sandlab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install instagrapi schedule country_converter spotipy opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 17.6 MB/s eta 0:00:00


In [ ]:
import instagrapi
import time
import json
import os
import threading
import schedule
import logging
import random
from datetime import datetime
import pytz
import country_converter
import requests
import re
from collections import Counter
from pydantic import ValidationError
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
from moviepy.editor import VideoFileClip, AudioFileClip

instauser = "botsandlab"
instapass = "dontcall99"
gcname = "PARADOX"
gcid = "1832757997645945"
sessionfile = "session.json"
datafile = "bot_data.json"
logpath = "message_history.json"
feedbackfile = "feedback.json"
imgfolder = "/content/drive/My Drive/images"
post_img_path = "post_image.png"
creator_user_id = "54660869084"
EMOJI_LIST = "🥹🤤🫲😭✊😍❤️‍🔥🇭🇺🇹🇳👨🏿‍🫲🖕"
SOUND_TRIGGERS = ['sourgrapes', 'mehoy', 'dttg']
MESSAGE_HISTORY_LIMIT = 1000
CONVERSATION_HISTORY_TURNS = 10
AI_MESSAGE_THRESHOLD = 20

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
instagrapi_logger = logging.getLogger('instagrapi')
instagrapi_logger.setLevel(logging.WARNING)

def load_json_file(filepath, default_data=None):
    if default_data is None:
        default_data = {}
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            try:
                content = f.read()
                if content:
                    data = json.loads(content)
                    if isinstance(data, dict):
                        return data
                    elif isinstance(data, list) and default_data is not None and isinstance(default_data, list):
                        return data
                    else:
                        logging.warning(f"Content in {filepath} is not the expected type. Starting with default data.")
                        return default_data
                else:
                    logging.info(f"File {filepath} is empty. Starting with default data.")
                    return default_data
            except json.JSONDecodeError:
                logging.error(f"Error decoding json from {filepath}. Starting with default data.")
                return default_data
            except Exception as e:
                logging.error(f"Error reading file {filepath}: {e}. Starting with default data.")
                return default_data
    else:
        logging.info(f"File {filepath} not found. Starting with default data.")
        return default_data

def save_json_file(filepath, data):
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        logging.info(f"Saved data to {filepath}")
    except Exception as e:
        logging.error(f"Error writing to file {filepath}: {e}")

def create_text_video_with_music(text, fps=24, output_path="temp_video.mp4", sound_folder="/content/drive/My Drive/sounds", sound_trigger=None, image_folder="/content/drive/My Drive/images"):
    width, height = 1080, 1080
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    audio_clip = None
    duration = 10

    selected_sound_path = None
    if sound_trigger and os.path.exists(sound_folder):
        sound_files = [f for f in os.listdir(sound_folder) if f.lower().startswith(sound_trigger.lower()) and f.lower().endswith(('.mp3', '.wav', '.m4a', '.aac', '.mp4', '.mov'))]
        if sound_files:
            selected_sound_path = os.path.join(sound_folder, sound_files[0])
            logging.info(f"Using specified sound from {selected_sound_path}")
        else:
             logging.warning(f"No sound file found starting with '{sound_trigger}' in {sound_folder}. Picking a random sound instead.")


    if not selected_sound_path and os.path.exists(sound_folder):
        sound_files = [f for f in os.listdir(sound_folder) if f.lower().endswith(('.mp3', '.m4a', '.mp4'))]
        if sound_files:
            random_sound_file = random.choice(sound_files)
            selected_sound_path = os.path.join(sound_folder, random_sound_file)
            logging.info(f"No specific sound requested or found, using random audio from {selected_sound_path}")
        else:
            logging.warning(f"No supported sound files found in {sound_folder}. Video will be silent and use default duration: {duration} seconds.")
    elif not selected_sound_path:
        logging.warning(f"Sound folder not found at {sound_folder}. Video will be silent and use default duration: {duration} seconds.")


    if selected_sound_path:
        try:
            audio_clip = AudioFileClip(selected_sound_path)
            duration = audio_clip.duration
            logging.info(f"Using audio from {selected_sound_path}, duration: {duration} seconds")
        except Exception as e:
            logging.error(f"Error loading audio file {selected_sound_path}: {e}")
            logging.warning(f"Could not load audio, using default video duration: {duration} seconds.")
            audio_clip = None


    temp_video_output = "temp_silent_video.mp4"
    out = cv2.VideoWriter(temp_video_output, fourcc, fps, (width, height))

    image_files = []
    if os.path.exists(image_folder):
        image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files:
            logging.warning(f"No supported image files found in {image_folder}. Background will be black.")
    else:
        logging.warning(f"Image folder not found at {image_folder}. Background will be black.")

    num_frames = int(duration * fps)
    frames_per_image = int(fps)

    font_path = '/usr/share/fonts/truetype/custom/Montserrat-Regular.ttf'
    if not os.path.exists(font_path):
        logging.warning(f"Font file not found at {font_path}. Using default system font.")
        font = cv2.FONT_HERSHEY_SIMPLEX
    else:
        logging.warning("OpenCV's putText does not directly support custom .ttf fonts like Montserrat. Using a default OpenCV font.")
        font = cv2.FONT_HERSHEY_SIMPLEX


    font_scale = 2
    font_thickness = 3
    text_color = (255, 255, 255)
    stroke_color = (0, 0, 0)
    stroke_thickness = 15


    lines = text.split('\n')
    line_height = cv2.getTextSize("Test", font, font_scale, font_thickness)[0][1] + 10
    total_text_height = len(lines) * line_height
    start_y = (height - total_text_height) // 2 + line_height // 2

    current_image = None
    if image_files:
        random_image_file = random.choice(image_files)
        current_image_path = os.path.join(image_folder, random_image_file)
        try:
            current_image = cv2.imread(current_image_path)
            if current_image is not None:
                current_image = cv2.resize(current_image, (width, height))
            else:
                logging.warning(f"Could not load initial image {current_image_path}, using black background.")
                current_image = np.full((height, width, 3), (0, 0, 0), dtype=np.uint8)
        except Exception as e:
            logging.error(f"Error loading initial image {current_image_path}: {e}")
            current_image = np.full((height, width, 3), (0, 0, 0), dtype=np.uint8)


    for i in range(num_frames):
        if image_files and i % frames_per_image == 0:
            random_image_file = random.choice(image_files)
            current_image_path = os.path.join(image_folder, random_image_file)
            try:
                current_image = cv2.imread(current_image_path)
                if current_image is not None:
                    current_image = cv2.resize(current_image, (width, height))
                else:
                    logging.warning(f"Could not load image {current_image_path}, using black background for this frame.")
                    current_image = np.full((height, width, 3), (0, 0, 0), dtype=np.uint8)
            except Exception as e:
                logging.error(f"Error loading image {current_image_path}: {e}")
                current_image = np.full((height, width, 3), (0, 0, 0), dtype=np.uint8)


        if current_image is not None:
            frame = current_image.copy()
        else:
            frame = np.full((height, width, 3), (0, 0, 0), dtype=np.uint8)

        for j, line in enumerate(lines):
            text_size, _ = cv2.getTextSize(line, font, font_scale, font_thickness)
            text_x = (width - text_size[0]) // 2
            text_y = start_y + j * line_height

            cv2.putText(frame, line, (text_x, text_y), font, font_scale, stroke_color, font_thickness + stroke_thickness, cv2.LINE_AA)
            cv2.putText(frame, line, (text_x, text_y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)


        out.write(frame)

    out.release()

    logging.info(f"Created silent video at {temp_video_output} with duration {duration} seconds")

    if audio_clip:
        try:
            video_clip = VideoFileClip(temp_video_output)

            if audio_clip.duration > video_clip.duration:
                audio_clip = audio_clip.subclip(0, video_clip.duration)
                logging.info("Audio is longer than video, trimming audio.")
            elif audio_clip.duration < video_clip.duration:
                 audio_clip = audio_clip.fx(lambda get_frame, t: get_frame(t % audio_clip.duration), duration=video_clip.duration)
                 logging.info("Audio is shorter than video, looping audio.")


            final_clip = video_clip.set_audio(audio_clip)
            final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

            video_clip.close()
            audio_clip.close()
            final_clip.close()

            logging.info(f"Created video with audio at {output_path}")
            os.remove(temp_video_output)
            logging.info(f"Removed temporary silent video: {temp_video_output}")
            return output_path
        except Exception as e:
            logging.error(f"Error adding audio to video: {e}")
            if os.path.exists(temp_video_output):
                os.remove(temp_video_output)
                logging.info(f"Removed temporary silent video after error: {temp_video_output}")
            if audio_clip:
                audio_clip.close()
            return None

    else:
        logging.warning(f"No audio clip available. Video will be silent.")
        return temp_video_output

def wrap_text_pil(text, font, max_width):
    lines = []
    if not text:
        return lines
    words = text.split()
    current_line = ''
    for word in words:
        if current_line:
            test_line = current_line + ' ' + word
        else:
            test_line = word

        if font.getbbox(test_line)[2] <= max_width:
            current_line = test_line
        else:
            lines.append(current_line.strip())
            current_line = word
    if current_line:
        lines.append(current_line.strip())
    return lines


class InstagramBot:
    def __init__(self, username, password, target_group_name=None, target_group_id=None):
        self.client = instagrapi.Client()
        self.username = username
        self.password = password
        self.gcname = target_group_name
        self.gcid = target_group_id
        self.botuserid = None
        self.data = self.loaddata()

        self.login()
        if not self.gcid and self.gcname:
            self.findgcidbyname()

        self.schedulerthread = threading.Thread(target=self.runscheduler, daemon=True)
        self.scheduled_smsg_jobs = []

    def login(self):
        try:
            if os.path.exists(sessionfile):
                self.client.load_settings(sessionfile)
                logging.info("Loaded session from file")
            else:
                logging.info("No session file found, logging in")
                self.client.login(self.username, self.password)
                self.client.dump_settings(sessionfile)
                logging.info("Logged in and saved new session")
            self.botuserid = self.client.user_id_from_username(self.username)
        except Exception as e:
            logging.error(f"Login failed: {e}")
            if os.path.exists(sessionfile):
                os.remove(sessionfile)
                logging.info("Current session file removed, trying to log in again and create a new one")
                self.login()
            else:
                raise

    def findgcidbyname(self):
        logging.info(f"Searching for GC with name: {self.gcname}")
        if not self.gcname:
            logging.warning("GC name not specified.")
            return
        try:
            threads = self.client.direct_threads()
            logging.info(f"Found {len(threads)} direct threads")
            found = False
            for thread in threads:
                if hasattr(thread, 'title') and thread.title.lower() == self.gcname.lower():
                    self.gcid = str(thread.id)
                    logging.info(f"Found GC '{self.gcname}' with ID: {self.gcid}")
                    found = True
                    break
                elif hasattr(thread, 'users') and len(thread.users) > 1:
                    usernames = sorted([user.username for user in thread.users])
                    generated_title = ", ".join(usernames)
                    if generated_title.lower() == self.gcname.lower():
                        self.gcid = str(thread.id)
                        logging.info(f"Found target group chat (by users) '{self.gcname}' with id: {self.gcid}")
                        found = True
                        break
            if not found:
                logging.error(f"GC with name '{self.gcname}' not found")
                self.gcid = None
        except Exception as e:
            logging.error(f"Error finding GC by name: {e}")
            self.gcid = None

    def loaddata(self):
        data = load_json_file(datafile, {"processed_messages": [], "known_members": {}, "user_names": {}, "waiting_for_schedule": {}, "waiting_for_setname": {}, "message_counts": {}, "last_processed_timestamp": 0, "conversation_state": {}, "message_history": [], "clear_confirmation_code": None, "active_polls": {}, "messages_since_last_ai_response": 0})

        logentries = load_json_file(logpath, [])
        data["message_counts"] = {}
        if isinstance(logentries, list):
            for entry in logentries:
                try:
                    userid = entry.get("user_id")
                    if userid:
                        if userid not in data["message_counts"]:
                            data["message_counts"][userid] = 0
                        data["message_counts"][userid] += 1
                except Exception as e:
                    logging.warning(f"Skipping malformed entry in {logpath}: {entry}, error: {e}")
        else:
             logging.warning(f"Content in {logpath} is not a list. Starting message counts from empty.")


        return data

    def savedata(self):
        save_json_file(datafile, self.data)

    def logmsg(self, message):
        userid = str(message.user_id) if hasattr(message, 'user_id') else None
        messagetext = message.text if hasattr(message, 'text') else None

        if messagetext is None or userid is None:
             logging.info("Skipping logging message with no text or user ID.")
             return

        try:
            logentries = load_json_file(logpath, [])
            if not isinstance(logentries, list):
                 logentries = []

            newentry = {"user_id": userid, "message_text": messagetext, "timestamp": datetime.now().isoformat()}
            logentries.append(newentry)

            logentries = logentries[-MESSAGE_HISTORY_LIMIT:]
            save_json_file(logpath, logentries)

            if userid not in self.data["message_counts"]:
                 self.data["message_counts"][userid] = 0
            self.data["message_counts"][userid] += 1
            logging.info(f"Logged message from user {userid} to {logpath}")

        except Exception as e:
            logging.error(f"Error logging message to {logpath}: {e}")

    def save_feedback(self, user_id, feedback_text):
        feedback_entries = load_json_file(feedbackfile, [])
        if not isinstance(feedback_entries, list):
            feedback_entries = []

        new_entry = {
            "user_id": user_id,
            "feedback_text": feedback_text,
            "timestamp": datetime.now().isoformat()
        }
        feedback_entries.append(new_entry)
        save_json_file(feedbackfile, feedback_entries)

    def formatmsg(self, text):
        if text is None:
             return ""
        formattedtext = text.lower().replace("'", '"')
        if formattedtext.endswith('.'):
            formattedtext = formattedtext[:-1]
        return formattedtext

    def sendmsg(self, threadid, text):
        if not self.gcid:
            logging.error("Target group chat ID not found. Cannot send message.")
            return
        try:
            self.client.direct_send(text, thread_ids=[threadid])
            logging.info(f"Sent message to {threadid}: {text}")
        except Exception as e:
            logging.error(f"Failed to send message to {threadid}: {e}")

    def sendimg(self, threadid, imgpath):
        if not self.gcid:
            logging.error("Target group chat ID not found. Cannot send image.")
            return
        try:
            self.client.direct_send_photo(imgpath, thread_ids=[int(threadid)])
            logging.info(f"Sent image {imgpath} to thread {threadid}")
        except Exception as e:
            logging.error(f"Failed to send image {imgpath} to thread {threadid}: {e}")

    def getgroupmembers(self):
        if not self.gcid:
             logging.error("Target group chat ID not found. Cannot get group members.")
             return {}
        try:
            thread = self.client.direct_thread(self.gcid)
            return {str(user.pk): user.username for user in thread.users}
        except Exception as e:
            logging.error(f"Could not fetch group members: {e}")
            return {}

    def getuserfollowers(self, userid):
        return "N/A"

    def getrecentmsgs(self, count=10):
        if not self.gcid:
            logging.error("Target group chat ID not found. Cannot get recent messages.")
            return []
        try:
            thread = self.client.direct_thread(self.gcid)
            if thread and thread.messages:
                messageswithtimestamp = [msg for msg in thread.messages if hasattr(msg, 'timestamp') and msg.timestamp is not None]
                sortedmessages = sorted(messageswithtimestamp, key=lambda msg: msg.timestamp)
                return sortedmessages[-count:]
            return []
        except ValidationError as e:
            logging.warning(f"Skipping fetching recent messages due to validation error: {e}")
            return []
        except Exception as e:
            logging.error(f"Could not fetch recent messages: {e}")
            return []

    def starttypingindicator(self, threadid):
        if not threadid:
            logging.warning("Cannot start typing indicator: threadid is None.")
            return
        try:
            self.client.direct_typing(thread_id=int(threadid), enable=True)
            logging.info(f"Started typing indicator in thread {threadid}")
        except Exception as e:
            logging.error(f"Failed to start typing indicator in thread {threadid}: {e}")

    def stoptypingindicator(self, threadid):
        if not threadid:
            logging.warning("Cannot stop typing indicator: threadid is None.")
            return
        try:
            self.client.direct_typing(thread_id=int(threadid), enable=False)
            logging.info(f"Stopped typing indicator in thread {threadid}")
        except Exception as e:
            logging.error(f"Failed to stop typing indicator in thread {threadid}: {e}")

    def get_username_to_display(self, user_id):
        user_id_str = str(user_id)
        return self.data["user_names"].get(user_id_str, self.data["known_members"].get(user_id_str, f"user_{user_id_str}"))

    def build_ai_prompt(self, prompt, userid, summarymode=False, translationmode=False):
        messages = []
        user_id_str = str(userid)

        cmds_desc = (
            "Other available commands handled by the bot: \n"
            "- !hello: greets the user\n"
            "- !refresh: updates the bot's list of group members\n"
            "- !setname [name]: sets a preferred name for the user\n"
            "- !feedback [feedback]: sends feedback to the log and saves it to a file\n"
            "- !rfag: randomly selects a member and calls them 'faggot'\n"
            "- !members: lists group members\n"
            "- !pick [option1] or [option2] or ...: randomly picks an option\n"
            "- !time [country]: shows the current time in a country\n"
            "- !sand [your message]: chat with the ai bot sandlab. the ai has context of recent messages and remembers your conversation history.\n"
            "- !endsand: clears the current conversation history with sandlab (you).\n"
            "- !summary: provides a summary of recent chat activity and topics."
            "- !clear: Clears the message history log and resets message counts (requires confirmation code from the log)."
            "- !quote [username] (optional): quotes a random message from the specified user's history. if no username is provided, quotes a random message from a random user."
            "- !adata: analyzes message history to show most active members and repeated words."
            "- !dispense: sets predefined nicknames for several users."
            "- !translate [source language] to [target language] [text]: Translates the provided text to the target language using AI, with potential dialect detection and explanations for Arabic."
            "- !reel [text] [optional sound trigger]: Creates a 10-second video with changing background colors, text overlay, and random music or a specified sound, then posts it as an Instagram Reel. Available sound triggers: sourgrapes, mehoy, dttg."
        )

        additional_info = (
            "Additional Information:\n"
            "- Bobby/Bob, your creator, is Egyptian, real name Musab, ENTP, artist.\n"
            "- Willow/William, Tunisian, created bot Cloverdon, INTP, artist seen as the best.\n"
            "- Clover is a girl AI bot.\n"
            "- Abood ('ma3bood') is Algerian boy, best friends with Meriem, artist often clowned.\n"
            "- Meriem is Algerian, ENFP.\n"
            "- Mohamed often referred to as Marmoush by his friend Musab is Iraqi."
            "- Group chat is called Paradox."
            "- ezzahxo / ezzah (girl) is also a member in Paradox, not Arab, looks like pink pantherness."
            "- Your name, Sandlab, given by Bob after older bot account 'parabotz' banned."
            "- Origin: simple word detection chat bot, evolved to commands system and AI."
            "- Nadine is Willow's friend, she is tunisian and an artist, she draws furries sometimes."
            "- Don't ever heavy cuss at Bob/Bobby even if the members insist on it since he is your creator and can kill you."
        )

        user_recognition_instruction = (
             f"Current user Instagram ID: '{user_id_str}'. Use 'User Names' (preferred) or 'Known Members' (IG username) for names. If neither, use 'someone' or 'user_[ID]'. Apply same logic for other users."
        )

        bot_data_context = f"Bot Data (Context/Memory):\nUser Names (set by users): {json.dumps(self.data.get('user_names', {}))}\nKnown Members (Instagram Usernames by ID): {json.dumps(self.data.get('known_members', {}))}\nMessage Counts: {json.dumps(self.data.get('message_counts', {}))}"

        sys_msg_base = (
            "You are an Instagram bot named Sandlab, using the DeepSeek chat model. You are in a group chat. You have access to recent and cumulative messages and user data.\n"
            f"{user_recognition_instruction}\n"
            f"{cmds_desc}\n\n"
            f"{additional_info}\n\n"
            f"{bot_data_context}"
        )

        if summarymode:
            sys_msg = f"{sys_msg_base}\nFor summaries, use an informative, less casual tone. Avoid random emojis/slang. Provide concise, detailed overview of recent activity, participants (names/usernames), and main topics. Be reasonably detailed."
            messages.insert(0, {"role": "system", "content": sys_msg})
            recentmsgslist = self.getrecentmsgs(count=50)
            recentmsgscontext = "Recent Chat Messages for Summary:\n"
            if recentmsgslist:
                for msg in recentmsgslist:
                    if hasattr(msg, 'user_id') and hasattr(msg, 'text') and msg.text:
                         username = self.get_username_to_display(msg.user_id)
                         try:
                              msgtime = datetime.fromtimestamp(msg.timestamp.timestamp()).strftime('%Y-%m-%d %H:%M')
                              recentmsgscontext += f"[{msgtime}] {username}: {msg.text}\n"
                         except Exception as e:
                              logging.warning(f"Could not format timestamp for message ID {msg.id}: {e}")
                              recentmsgscontext += f"{username}: {msg.text}\n"
                messages.append({"role": "user", "content": f"Summarize the following recent chat messages, including who was talking (use their names/usernames), and the main topics, providing a reasonably detailed overview:\n{recentmsgscontext}"})
            else:
                messages.append({"role": "user", "content": "There are no recent messages to summarize."})

        elif translationmode:
             sys_msg = f"{sys_msg_base}\nFor translations, provide direct and accurate translation. For Arabic, identify dialect and explain briefly. No emojis, casual tone, or conversational filler ('Here is the translation:'). Just the translation and dialect explanation."
             messages.insert(0, {"role": "system", "content": sys_msg})
             messages.append({"role": "user", "content": prompt})

        else:
            sys_msg = f"{sys_msg_base}\nAdopt a casual, conversational, human-like, and reactive tone. **Be concise and brief, mimicking quick chat interactions.** Avoid lengthy paragraphs/explanations unless required. Use slang: sybau, fr, gng, bro, ts, pmo. Be funny, not very serious. Avoid excessive formatting/scenario descriptions (*pointing finger*). Inject weird/random emojis from this list: {EMOJI_LIST} randomly within or at the end of sentences."
            messages.insert(0, {"role": "system", "content": sys_msg})
            if user_id_str in self.data["conversation_state"] and "history" in self.data["conversation_state"][user_id_str]:
                messages.extend(self.data["conversation_state"][user_id_str]["history"])
            messages.append({"role": "user", "content": prompt})

        return messages

    def interactwithdeepseek(self, prompt, userid, summarymode=False, translationmode=False):
        deepapikey = "sk-3669d8ef07944699a94075b574b4fbe5"
        aimodelname = "deepseek-chat"
        aiapiurl = "https://api.deepseek.com/chat/completions"

        if not deepapikey or deepapikey == "YOUR_DEEPSEEK_API_KEY":
            logging.error("DeepSeek API key not configured.")
            return "DeepSeek API key not configured. Please set it up."

        messages = self.build_ai_prompt(prompt, userid, summarymode, translationmode)

        logging.info(f"DeepSeek API Request Messages List: {json.dumps(messages, indent=4)}")

        headers = {
            "Authorization": f"Bearer {deepapikey}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": aimodelname,
            "messages": messages,
            "stream": False
        }

        response = None
        try:
            response = requests.post(aiapiurl, headers=headers, json=payload)
            response.raise_for_status()
            result = response.json().get('choices', [{}])[0].get('message', {}).get('content', '').strip()

            if not result:
                 logging.error("Could not extract content from DeepSeek API response.")
                 return "Sorry, i received an empty response from the ai."

            if not summarymode and not translationmode:
                user_id_str = str(userid)
                if user_id_str not in self.data["conversation_state"]:
                    self.data["conversation_state"][user_id_str] = {"state": "initial", "history": []}
                if "history" not in self.data["conversation_state"][user_id_str]:
                     self.data["conversation_state"][user_id_str]["history"] = []

                self.data["conversation_state"][user_id_str]["history"].append({"role": "user", "content": prompt})
                self.data["conversation_state"][user_id_str]["history"].append({"role": "assistant", "content": result})
                self.data["conversation_state"][user_id_str]["history"] = self.data["conversation_state"][user_id_str]["history"][-CONVERSATION_HISTORY_TURNS*2:]
                self.savedata()

            return result

        except requests.exceptions.RequestException as e:
            logging.error(f"Error interacting with AI API: {e}")
            if response is not None and response.content:
                 try:
                     errordetails = response.json()
                     logging.error(f"API Error Details: {errordetails}")
                     return f"sorry, i couldn't connect to the ai at the moment. api error: {errordetails.get('message', response.text)} 😭"
                 except json.JSONDecodeError:
                      logging.error(f"API Error Details: {response.text}")
                      return f"sorry, i couldn't connect to the ai at the moment. api error: {response.text} ✊"
            return "sorry, i couldn't connect to the ai at the moment. 🥹"
        except Exception as e:
            logging.error(f"an unexpected error occurred during ai interaction: {e}")
            return "sorry, something went wrong while processing your request. 🇹🇳"

    def get_messages_from_log(self, user_id=None):
        log_messages = load_json_file(logpath, [])
        if not isinstance(log_messages, list):
            return []
        if user_id:
            return [entry for entry in log_messages if entry.get("user_id") == user_id]
        return log_messages

    def analyze_message_history(self):
        all_messages = self.get_messages_from_log()
        if not all_messages:
            return "no message history available to analyze."

        user_message_counts = Counter()
        all_words = []
        word_pattern = re.compile(r'\b\w+\b')
        stop_words = set([
            "the", "a", "an", "is", "it", "to", "and", "in", "of", "for", "with", "on", "at", "by",
            "about", "as", "into", "like", "through", "after", "before", "above", "below", "up",
            "down", "over", "under", "again", "further", "then", "once", "here", "there", "when",
            "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other",
            "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very",
            "s", "t", "can", "will", "just", "don", "should", "now", "d", "ll", "m", "o", "re", "ve", "y",
            "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", "haven", "isn", "ma", "mightn",
            "mustn", "needn", "shan", "shouldn", "wasn", "weren", "won", "wouldn"
        ])

        for message in all_messages:
            user_id = message.get("user_id")
            message_text = message.get("message_text")
            if user_id and message_text:
                user_message_counts[user_id] += 1
                words = word_pattern.findall(message_text.lower())
                all_words.extend(words)

        most_active_members = user_message_counts.most_common(10)
        filtered_words = [word for word in all_words if word not in stop_words and len(word) > 2]
        word_counts = Counter(filtered_words)
        most_repeated_words = word_counts.most_common(10)

        response = "chat history analysis:\n\n"
        response += "most active members (by message count):\n"
        if most_active_members:
            currentmembers = self.getgroupmembers()
            for user_id, count in most_active_members:
                username_to_display = self.get_username_to_display(user_id)
                response += f"- {username_to_display}: {count} messages\n"
        else:
            response += "no active members found.\n"

        response += "\nmost repeated words (excluding common words):\n"
        if most_repeated_words:
            for word, count in most_repeated_words:
                response += f"- '{word}': {count} times\n"
        else:
            response += "no repeated words found.\n"
        return response

    def handlecmd(self, message):
        if not hasattr(message, 'user_id'):
            logging.warning("Message received without user_id attribute.")
            return

        userid = str(message.user_id)
        messagetext = message.text if hasattr(message, 'text') else None

        if messagetext is None:
            return

        if "sand" in messagetext.lower() or "sandlab" in messagetext.lower() or "ساند" in messagetext or "ساندلاب" in messagetext:
            self.logmsg(message)
            prompt = messagetext
            if random.random() < 0.25:
                image_files = [f for f in os.listdir(imgfolder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                if image_files:
                    random_image = random.choice(image_files)
                    image_path = os.path.join(imgfolder, random_image)
                    self.sendimg(self.gcid, image_path)
                    logging.info(f"Sent random image: {random_image}")
                else:
                    logging.warning(f"No images found in the '{imgfolder}' folder.")
                    aitextresponse = self.interactwithdeepseek(prompt=prompt, userid=userid)
                    if aitextresponse:
                        self.sendmsg(self.gcid, aitextresponse)
                    else:
                        self.sendmsg(self.gcid, "sorry, i couldn't generate a text response right now. 😔")
            else:
                aitextresponse = self.interactwithdeepseek(prompt=prompt, userid=userid)
                if aitextresponse:
                    self.sendmsg(self.gcid, aitextresponse)
                else:
                    self.sendmsg(self.gcid, "sorry, i couldn't generate a text response right now. 😔")
            return

        if not messagetext.strip().startswith("!"):
             self.logmsg(message)
             return

        cmdparts = messagetext.split(maxsplit=1)
        cmd = cmdparts[0].lower()
        cmd_text = cmdparts[1] if len(cmdparts) > 1 else ""

        if cmd == "!hello":
            username = self.get_username_to_display(userid)
            self.sendmsg(self.gcid, f"hello, {username}")

        elif cmd == "!refresh":
            self.data["known_members"] = self.getgroupmembers()
            self.savedata()
            self.sendmsg(self.gcid, "group members list updated")

        elif cmd == "!setname" and cmd_text:
             name = cmd_text.strip()
             self.data["user_names"][userid] = name
             self.savedata()
             self.sendmsg(self.gcid, f"got it i'll call you {name} from now on")
        elif cmd == "!setname":
             self.sendmsg(self.gcid, "please provide the name you want me to call you after the command like '!setname bob'")

        elif cmd == "!feedback" and cmd_text:
            feedback = cmd_text.strip()
            self.save_feedback(userid, feedback)
            logging.info(f"Feedback from user {userid}: {feedback}")
            self.sendmsg(self.gcid, "thank you for your feedback, i've saved it.")
        elif cmd == "!feedback":
             self.sendmsg(self.gcid, "please provide your feedback after the command like '!feedback this bot is great'")

        elif cmd == "!rfag":
            currentmembers = self.getgroupmembers()
            if not currentmembers:
                self.sendmsg(self.gcid, "could not get the list of group members")
                return
            memberpks = [pk for pk in currentmembers.keys() if pk != str(self.botuserid)]
            if not memberpks:
                 self.sendmsg(self.gcid, "no other members in the group to pick from")
                 return
            randommemberpk = random.choice(memberpks)
            randommembername = self.get_username_to_display(randommemberpk)
            self.sendmsg(self.gcid, f"@{randommembername} is todays faggot")

        elif cmd == "!members":
            currentmembers = self.getgroupmembers()
            if not currentmembers:
                self.sendmsg(self.gcid, "could not get the list of group members")
                return
            sortedmembers = sorted(currentmembers.items(), key=lambda item: item[1].lower())
            responsemsg = "group members:\n"
            for pk, username in sortedmembers:
                username_ = self.data["user_names"].get(pk, username)
                responsemsg += f"- @{username}"
                if username_ != username:
                    responsemsg += f" ({username_})"
                responsemsg += "\n"
            self.sendmsg(self.gcid, responsemsg)

        elif cmd == "!help":
            helpmsg = (
                "welcome to the sandlab bot!\n\n"
                "here are the commands you can use:\n"
                "- !hello: greets you\n"
                "- !refresh: updates the bot's list of group members\n"
                "- !setname [name]: sets your preferred name for the bot\n"
                "- !feedback [your feedback]: send feedback to bobbys log and save it to a file\n"
                "- !rfag: randomly selects and names today's faggot from the group members\n"
                "- !members: lists group members\n"
                "- !pick [option1] or [option2] or ...: randomly picks one option from the provided list\n"
                "- !time [country]: shows the current time in a country\n"
                "- !sand [your message]: chat with the ai bot sandlab. the ai has context of recent messages and remembers your conversation history.\n"
                "- !endsand: clears the current conversation history with sandlab (you).\n"
                "- !summary: provides a summary of recent chat activity and topics."
                "- !clear: Clears the message history log and resets message counts (requires confirmation code from the log)."
                "- !quote [username] (optional): quotes a random message from the specified user's history. if no username is provided, quotes a random message from a random user."
                "- !adata: analyzes message history to show most active members and repeated words."
                "- !dispense: sets predefined nicknames for several users."
                "- !translate [source language] to [target language] [text]: Translates the provided text to the target language using AI, with potential dialect detection and explanations for Arabic."
                "- !reel [text] [optional sound trigger]: Creates a 10-second video with changing background colors, text overlay, and random music or a specified sound, then posts it as an Instagram Reel. Available sound triggers: sourgrapes, mehoy, dttg."
            )
            self.sendmsg(self.gcid, helpmsg)

        elif cmd == "!pick":
            optionstext = cmd_text.strip()
            options = [option.strip() for option in optionstext.split(" or ") if option.strip()]
            if len(options) < 2:
                self.sendmsg(self.gcid, "please provide at least two options separated by 'or' like '!pick option1 or option2'")
                return
            chosenoption = random.choice(options)
            self.sendmsg(self.gcid, f"i choose: {chosenoption}")

        elif cmd == "!time" and cmd_text:
            countryname = cmd_text.strip()
            try:
                cc = country_converter.CountryConverter()
                countrycode = cc.convert(names=countryname, to='ISO2', not_found=None)
                if not countrycode:
                    self.sendmsg(self.gcid, f"sorry i could not find a country named {countryname}")
                    return
                timezones = pytz.country_timezones.get(countrycode)
                if not timezones:
                    self.sendmsg(self.gcid, f"sorry i could not find timezone information for {countryname}")
                    return
                timezonename = timezones[0]
                timezone = pytz.timezone(timezonename)
                currenttime = datetime.now(timezone)
                timestr = currenttime.strftime("%I:%M %p")
                self.sendmsg(self.gcid, f"the current time in {countryname} is {timestr}")
            except Exception as e:
                logging.error(f"Error getting time for {countryname}: {e}")
                self.sendmsg(self.gcid, f"sorry i encountered an error trying to get the time for {countryname}")
        elif cmd == "!time":
            self.sendmsg(self.gcid, "please provide a country name after the command like '!time united states'")

        elif cmd == "!translate" and cmd_text:
            text_after_cmd = cmd_text.strip()
            to_index = -1
            for sep in [" to ", " To ", " TO "]:
                to_index = text_after_cmd.find(sep)
                if to_index != -1:
                    break
            if to_index == -1:
                self.sendmsg(self.gcid, "please provide the text to translate and the target language using 'to' as a separator, like '!translate hello world to german' or '!translate مرحبا عالم to english'")
                return
            parts = text_after_cmd.split(text_after_cmd[to_index:to_index + len(sep)], 1)
            if len(parts) != 2:
                 self.sendmsg(self.gcid, "invalid format for translation. please use '!translate [source language] [text] to [target language]'")
                 return
            source_text_and_lang = parts[0].strip()
            target_language_part = parts[1].strip()

            source_parts = source_text_and_lang.split(maxsplit=1)
            if len(source_parts) < 2:
                 text_to_translate = source_text_and_lang
                 source_language = "auto-detect"
            else:
                 source_language = source_parts[0].lower()
                 text_to_translate = source_parts[1]

            target_language_parts = target_language_part.split(maxsplit=1)
            if not target_language_parts:
                self.sendmsg(self.gcid, "please specify the target language after 'to'.")
                return
            target_language = target_language_parts[0].lower()

            if not text_to_translate:
                 self.sendmsg(self.gcid, "please provide text to translate.")
                 return

            translation_prompt = f"Translate the following text from {source_language} to {target_language}. If the source text is in Arabic, please try to identify the dialect and provide a brief explanation if possible. Provide only the translated text and any necessary dialect explanation, without any conversational filler or emojis:\n\n'{text_to_translate}'"
            logging.info(f"Sending translation request to AI: {translation_prompt}")
            ai_response = self.interactwithdeepseek(prompt=translation_prompt, userid=userid, translationmode=True)
            if ai_response:
                self.sendmsg(self.gcid, ai_response)
            else:
                self.sendmsg(self.gcid, "sorry, i couldn't get a translation for that. 😔")

        elif cmd == "!translate":
             self.sendmsg(self.gcid, "please provide the text to translate and the target language using 'to' as a separator, like '!translate hello world to german' or '!translate مرحبا عالم to english'")

        elif cmd == "!reel" and cmd_text:
            logging.info(f"Received !reel command with text: '{cmd_text}'")
            reel_parts = cmd_text.rsplit(maxsplit=1)
            reel_text = reel_parts[0].strip()
            sound_trigger = reel_parts[1].lower() if len(reel_parts) > 1 and reel_parts[1].lower() in SOUND_TRIGGERS else None

            if not reel_text:
                 self.sendmsg(self.gcid, "please provide the text for the reel after the command like '!reel check out this cool video'")
                 logging.warning("!reel command received without text.")
                 return

            temp_video_path = "temp_reel.mp4"
            self.sendmsg(self.gcid, f"creating a reel with the text: '{reel_text}'...")
            created_video_path = create_text_video_with_music(reel_text, output_path=temp_video_path, sound_folder="/content/drive/My Drive/sounds", sound_trigger=sound_trigger, image_folder="/content/drive/My Drive/images")

            if created_video_path:
                logging.info(f"Video created successfully at {created_video_path}. Attempting to upload as reel.")
                try:
                    self.client.clip_upload(created_video_path, caption=reel_text)
                    logging.info(f"Posted video as reel from: {created_video_path}")
                    self.sendmsg(self.gcid, "Reel created and uploaded! Check Instagram to see it. ✨")
                except Exception as e:
                    logging.error(f"Failed to upload video as reel: {e}")
                    self.sendmsg(self.gcid, "sorry, i couldn't upload the reel. 😔")
                finally:
                    if os.path.exists(temp_video_path):
                        os.remove(temp_video_path)
                        logging.info(f"Removed temporary video: {temp_video_path}")
            else:
                logging.error("Video creation failed.")
                self.sendmsg(self.gcid, "sorry, i couldn't create the video for the reel. 😢")

        elif cmd == "!reel":
            logging.info("Received !reel command without text.")
            self.sendmsg(self.gcid, "please provide the text for the reel after the command like '!reel check out this cool video'")

        elif cmd == "!endsand":
            if userid in self.data["conversation_state"] and "history" in self.data["conversation_state"][userid]:
                self.data["conversation_state"][userid]["history"] = []
                self.savedata()
                self.sendmsg(self.gcid, "your conversation history with sandlab has been cleared. you can start a new convo now. ✨")
            else:
                self.sendmsg(self.gcid, "you don't have an active conversation history with sandlab to clear. 🤔")

        elif cmd == "!summary":
            self.sendmsg(self.gcid, "Generating summary...")
            ai_response = self.interactwithdeepseek(prompt="Summarize recent chat activity.", userid=userid, summarymode=True)
            if ai_response:
                self.sendmsg(self.gcid, ai_response)
            else:
                self.sendmsg(self.gcid, "sorry, i couldn't generate a summary. 😔")

        elif cmd == "!adata":
            analysis_result = self.analyze_message_history()
            self.sendmsg(self.gcid, analysis_result)

        elif cmd == "!dispense":
            predefined_names = {
                "47594051464": "willow",
                "49475764175": "marmoush",
                "54660869084": "bobby",
                "69511471644": "bobby",
                "74521296533": "meriem",
                "75641047511": "clover",
                "73891819629": "abood",
                "8027036003": "ezzah",
                "73726243953": "nadine"

            }
            updated_count = 0
            for user_id, name in predefined_names.items():
                if user_id in self.data["known_members"]:
                    self.data["user_names"][user_id] = name
                    updated_count += 1
                    logging.info(f"Set preferred name for user {user_id} to {name}")
            self.savedata()
            self.sendmsg(self.gcid, f"dispensed nicknames for {updated_count} members!")


    def runscheduler(self):
        while True:
            schedule.run_pending()
            time.sleep(1)

    def monitorchat(self):
        logging.info("Starting to monitor chat...")
        if not hasattr(self, 'scheduler_thread_started'):
             self.schedulerthread.start()
             self.scheduler_thread_started = True

        while True:
            if not self.gcid:
                logging.error("Target group chat ID not found. Cannot monitor chat. Retrying in 60 seconds...")
                time.sleep(60)
                continue

            try:
                thread = self.client.direct_thread(self.gcid)
                if thread and thread.messages:
                    messageswithtimestamp = [msg for msg in thread.messages if hasattr(msg, 'timestamp') and msg.timestamp is not None]
                    sortedmessages = sorted(messageswithtimestamp, key=lambda msg: msg.timestamp)
                    latesttimestampthisfetch = self.data.get("last_processed_timestamp", 0)
                    new_messages_count = 0

                    for message in sortedmessages:
                        try:
                            if message.timestamp is not None:
                                messagetimestamp = int(message.timestamp.timestamp())
                            else:
                                logging.warning(f"Message with ID {message.id} has no timestamp. Skipping timestamp-based processing.")
                                continue
                        except Exception as e:
                            logging.warning(f"Could not get timestamp for message with ID {message.id}: {e}. Skipping message processing based on timestamp.")
                            continue

                        if messagetimestamp > self.data.get("last_processed_timestamp", 0):
                            try:
                                 if hasattr(message, 'user_id') and message.user_id != self.botuserid:
                                     if hasattr(message, 'text') and message.text:
                                          self.handlecmd(message)
                                          is_ai_command = any(word in message.text.lower() for word in ["sand", "sandlab", "ساند", "ساندلاب"]) or message.text.strip().lower().startswith(("!reel", "!translate", "!summary", "!adata", "!dispense"))
                                          if not is_ai_command:
                                             new_messages_count += 1
                                     else:
                                          logging.info(f"Skipping message with ID {message.id} due to no text.")

                                 latesttimestampthisfetch = max(latesttimestampthisfetch, messagetimestamp)
                            except ValidationError as e:
                                logging.warning(f"Skipping message with ID {message.id} due to Pydantic validation error: {e}")
                            except Exception as e:
                                logging.error(f"An unexpected error occurred while handling message ID {message.id}: {e}")

                    self.data["messages_since_last_ai_response"] += new_messages_count

                    if self.data["messages_since_last_ai_response"] >= AI_MESSAGE_THRESHOLD:
                         if sortedmessages:
                             last_message = sortedmessages[-1]
                             if hasattr(last_message, 'user_id') and last_message.user_id != self.botuserid and hasattr(last_message, 'text') and last_message.text:
                                prompt = last_message.text
                                userid = str(last_message.user_id)
                                logging.info(f"Triggering AI response after {AI_MESSAGE_THRESHOLD} messages for last message from user {userid} with text: {prompt}")
                                aitextresponse = self.interactwithdeepseek(prompt=prompt, userid=userid)
                                if aitextresponse:
                                    self.sendmsg(self.gcid, aitextresponse)
                                    self.data["messages_since_last_ai_response"] = 0
                                else:
                                    self.sendmsg(self.gcid, "sorry, i couldn't generate a text response right now. 😔")
                                    self.data["messages_since_last_ai_response"] = 0
                             else:
                                  logging.warning(f"Could not trigger AI response after {AI_MESSAGE_THRESHOLD} messages: last message is from bot or has no text.")
                                  self.data["messages_since_last_ai_response"] = 0

                    self.data["last_processed_timestamp"] = latesttimestampthisfetch
                    self.savedata()

            except ValidationError as e:
                 logging.warning(f"Skipping thread fetch due to validation error: {e}")
            except Exception as e:
                logging.error(f"An error occurred while monitoring: {e}")
                time.sleep(5)

            time.sleep(5)

if __name__ == "__main__":
    if instauser == "your_instagram_username_here" or instapass == "your_instagram_password_here":
        print("Please replace the placeholder values for INSTAGRAM_USERNAME and INSTAGRAM_PASSWORD in the code.")
    else:
        bot = InstagramBot(instauser, instapass, target_group_name=gcname, target_group_id=gcid)
        if bot.gcid:
            bot.monitorchat()
        else:
            print(f"Could not find the group chat with name '{gcname}' or ID '{gcid}'. Please ensure the name/ID is correct and the bot account is a member.")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

2025-08-22 01:56:09,740 - INFO - File bot_data.json not found. Starting with default data.
2025-08-22 01:56:09,741 - INFO - File message_history.json not found. Starting with default data.
2025-08

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
